# 🌿 Ecological Meta-Analysis with `ecometa` Library

**Version 4.0** - Clean Implementation Using Professional Library

This notebook demonstrates the complete workflow for three-level meta-analysis using the `ecometa` Python library.

**Key Improvements over v3.0:**
- ✅ No function definitions needed (all in library)
- ✅ Clean, readable code
- ✅ Reproducible with `pip install -e .`
- ✅ Professional scikit-learn style API

---

## 📦 1. SETUP & IMPORTS

In [ ]:
# Standard libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# ecometa library - Core models
from ecometa import (
    ThreeLevelMeta,
    MetaRegression,
    SplineMetaRegression
)

# ecometa library - Effect size calculators
from ecometa import (
    calculate_hedges_g,
    calculate_lnrr
)

# ecometa library - Plotting
from ecometa.plots import (
    forest_plot,
    funnel_plot,
    orchard_plot,
    spline_plot,
    trim_and_fill_plot
)

# ecometa library - Interactive widgets
from ecometa.interactive.plot_guis import (
    orchard_plot_interactive,
    forest_plot_interactive,
    funnel_plot_interactive,
    spline_plot_interactive
)

# ecometa library - Utilities
from ecometa.utils import (
    leave_one_out_analysis,
    calculate_i2_3level,
    compare_models,
    interpret_heterogeneity
)

# Configure plotting
sns.set_style('whitegrid')
plt.rcParams['figure.dpi'] = 100

print("✅ All libraries imported successfully!")
print("📚 ecometa version: 0.1.0")

## 📊 2. DATA LOADING

Load your meta-analysis dataset. The dataset should contain:
- **Effect sizes** (e.g., Hedges' g, lnRR)
- **Sampling variances**
- **Study identifiers** (for grouping)
- **Moderator variables** (optional)

In [ ]:
# Option 1: Load from CSV
# data = pd.read_csv('your_data.csv')

# Option 2: Example dataset
# Create a sample dataset for demonstration
np.random.seed(42)

n_studies = 15
n_effects_per_study = [3, 2, 4, 2, 3, 2, 3, 2, 4, 3, 2, 3, 2, 3, 2]

data_list = []
for study_id in range(n_studies):
    n_effects = n_effects_per_study[study_id]
    study_mean = np.random.normal(0.5, 0.3)
    
    for effect_id in range(n_effects):
        # Generate effect sizes with between-study and within-study variation
        effect = study_mean + np.random.normal(0, 0.2)
        variance = np.random.uniform(0.05, 0.15)
        
        data_list.append({
            'study_id': f'Study_{study_id + 1}',
            'effect_id': effect_id + 1,
            'hedges_g': effect,
            'Vg': variance,
            'SE': np.sqrt(variance),
            'year': np.random.randint(2010, 2024),
            'temperature': np.random.uniform(15, 30),
            'ecosystem': np.random.choice(['Forest', 'Grassland', 'Wetland'])
        })

data_filtered = pd.DataFrame(data_list)

print(f"📈 Dataset loaded: {len(data_filtered)} effect sizes from {data_filtered['study_id'].nunique()} studies")
print(f"\nFirst few rows:")
data_filtered.head()

## 🧮 3. THREE-LEVEL META-ANALYSIS

Run the core meta-analysis using the `ThreeLevelMeta` class.

**That's it!** No need to define complex functions. The library handles:
- REML estimation
- Sherman-Morrison matrix inversion
- Multi-start optimization
- Confidence intervals
- Heterogeneity statistics

In [ ]:
# Initialize and fit the model (2 lines!)
model = ThreeLevelMeta(
    data=data_filtered,
    effect_col='hedges_g',
    var_col='Vg',
    study_id_col='study_id'
)

model.fit()

# Display results
print(model.summary())

In [ ]:
# Access results programmatically
results = model.results

print(f"\n📊 KEY RESULTS:")
print(f"Pooled Effect: {results['pooled_effect']:.3f} (95% CI: [{results['ci_lower']:.3f}, {results['ci_upper']:.3f}])")
print(f"P-value: {results['p_value']:.4f}")
print(f"\n🔍 HETEROGENEITY:")
print(f"τ² (between-study): {results['tau2']:.4f}")
print(f"σ² (within-study): {results['sigma2']:.4f}")

# Calculate I² statistics
typical_variance = data_filtered['Vg'].mean()
i2_stats = calculate_i2_3level(
    tau2=results['tau2'],
    sigma2=results['sigma2'],
    typical_variance=typical_variance
)

print(f"\nI² (total): {i2_stats['i2_total']:.1f}% - {interpret_heterogeneity(i2_stats['i2_total'])}")
print(f"I² (between-study): {i2_stats['i2_between']:.1f}%")
print(f"I² (within-study): {i2_stats['i2_within']:.1f}%")

## 🍎 4. ORCHARD PLOT (Interactive)

Visualize the meta-analysis results with an interactive orchard plot.

**Features:**
- Trunk: Prediction interval
- Fruit: Pooled effect with CI
- Leaves: Raw effect sizes
- Optional subgroup analysis

In [ ]:
# Interactive orchard plot with customization widgets
orchard_plot_interactive(model)

## 📊 5. FOREST PLOT (Static)

Create a publication-ready forest plot showing individual studies and overall effect.

In [ ]:
# Static forest plot for publication
fig, ax = forest_plot(
    data=data_filtered,
    effect_col='hedges_g',
    var_col='Vg',
    study_col='study_id',
    overall_effect=results['pooled_effect'],
    overall_se=results['se'],
    title="Forest Plot: Effect of Treatment on Outcome",
    xlabel="Hedges' g",
    color='steelblue',
    show_weights=True
)

plt.show()

In [ ]:
# Forest plot with subgroups
fig, ax = forest_plot(
    data=data_filtered,
    effect_col='hedges_g',
    var_col='Vg',
    study_col='study_id',
    group_col='ecosystem',  # Subgroup by ecosystem type
    overall_effect=results['pooled_effect'],
    overall_se=results['se'],
    title="Forest Plot by Ecosystem Type",
    xlabel="Hedges' g"
)

plt.show()

## 📈 6. META-REGRESSION

Investigate how moderator variables explain heterogeneity.

**Example:** Does the effect vary by publication year?

In [ ]:
# Initialize meta-regression model
reg_model = MetaRegression(
    data=data_filtered,
    effect_col='hedges_g',
    var_col='Vg',
    study_id_col='study_id',
    moderators='year'  # Can also pass list: ['year', 'temperature']
)

# Fit the model
reg_model.fit()

# Display results
print(reg_model.summary())

In [ ]:
# Access regression coefficients
reg_results = reg_model.results
coef_table = reg_results['coefficients']

print("\n📊 REGRESSION COEFFICIENTS:")
print(coef_table)

# Interpretation
year_coef = coef_table.loc['year', 'coefficient']
year_pval = coef_table.loc['year', 'p_value']

if year_pval < 0.05:
    print(f"\n✅ Publication year is a significant moderator (p = {year_pval:.4f})")
    print(f"   Effect changes by {year_coef:.4f} per year")
else:
    print(f"\n❌ Publication year is not a significant moderator (p = {year_pval:.4f})")

## 🌊 7. SPLINE META-REGRESSION

Explore non-linear relationships between continuous moderators and effect sizes.

**Example:** How does the effect vary with temperature (non-linearly)?

In [ ]:
# Initialize spline model
spline_model = SplineMetaRegression(
    data=data_filtered,
    effect_col='hedges_g',
    var_col='Vg',
    study_id_col='study_id',
    moderator_col='temperature',
    df_spline=4  # Degrees of freedom (controls smoothness)
)

# Fit the model
spline_model.fit()

# Display summary
print(spline_model.summary())

In [ ]:
# Generate predictions for plotting
temp_range = np.linspace(
    data_filtered['temperature'].min(),
    data_filtered['temperature'].max(),
    100
)

predictions = spline_model.predict(temp_range, ci=True)

# Create spline plot
fig, ax = spline_plot(
    moderator_values=predictions['moderator'],
    predictions=predictions['predicted_effect'],
    ci_lower=predictions['ci_lower'],
    ci_upper=predictions['ci_upper'],
    observed_effects=data_filtered['hedges_g'],
    observed_moderators=data_filtered['temperature'],
    title="Non-linear Effect of Temperature",
    xlabel="Temperature (°C)",
    ylabel="Hedges' g"
)

plt.show()

## 🔄 8. LEAVE-ONE-OUT SENSITIVITY ANALYSIS

Identify influential studies by refitting the model k times, each time excluding one study.

In [ ]:
# Run leave-one-out analysis
loo_results = leave_one_out_analysis(
    model_class=ThreeLevelMeta,
    data=data_filtered,
    effect_col='hedges_g',
    var_col='Vg',
    study_id_col='study_id'
)

print("\n🔍 MOST INFLUENTIAL STUDIES:")
print(loo_results.head(5))

# Plot influence
fig, ax = plt.subplots(figsize=(10, 6))

ax.barh(range(len(loo_results)), loo_results['influence'])
ax.set_yticks(range(len(loo_results)))
ax.set_yticklabels(loo_results['study_id'])
ax.set_xlabel('Influence (Absolute Change in Pooled Effect)', fontweight='bold')
ax.set_title('Leave-One-Out Influence Analysis', fontweight='bold', fontsize=14)
ax.invert_yaxis()

plt.tight_layout()
plt.show()

## 📉 9. PUBLICATION BIAS ASSESSMENT

Assess potential publication bias using funnel plots and Egger's test.

In [ ]:
# Create funnel plot
fig, ax = funnel_plot(
    effect_sizes=data_filtered['hedges_g'],
    variances=data_filtered['Vg'],
    center=results['pooled_effect'],
    title="Funnel Plot for Publication Bias",
    show_contours=True
)

plt.show()

print("\n👁️ VISUAL INSPECTION:")
print("A symmetric funnel suggests no publication bias.")
print("Asymmetry may indicate missing studies (e.g., unpublished negative results).")

In [ ]:
# Interactive funnel plot with customization
funnel_plot_interactive(
    effect_sizes=data_filtered['hedges_g'],
    variances=data_filtered['Vg'],
    center=results['pooled_effect']
)

## 🎯 10. MODEL COMPARISON

Compare different models using information criteria (AIC, BIC).

In [ ]:
# Prepare models dictionary
models = {
    'Overall': {
        'log_likelihood': model.results['log_likelihood'],
        'n_params': 3,  # mu, tau2, sigma2
        'n_obs': len(data_filtered)
    },
    'Regression (Year)': {
        'log_likelihood': reg_model.results['log_likelihood'],
        'n_params': reg_model.results['n_studies'] + 2,
        'n_obs': len(data_filtered)
    },
    'Spline (Temperature)': {
        'log_likelihood': -spline_model.results['aic'] / 2 - spline_model.results['wls_model'].nobs / 2,
        'n_params': len(spline_model.results['params']) + 1,
        'n_obs': len(spline_model.results['agg_data'])
    }
}

# Compare models
comparison = compare_models(models)

print("\n📊 MODEL COMPARISON:")
print(comparison)

print("\n✅ BEST MODEL (lowest AIC):")
best_model = comparison.iloc[0]['Model']
print(f"   {best_model}")

## 📝 11. EXPORT RESULTS

Save results for publication or further analysis.

In [ ]:
# Create results summary
summary_dict = {
    'Analysis': 'Three-Level Meta-Analysis',
    'N_Studies': model.n_studies,
    'N_Effects': model.n_obs,
    'Pooled_Effect': results['pooled_effect'],
    'SE': results['se'],
    'CI_Lower': results['ci_lower'],
    'CI_Upper': results['ci_upper'],
    'P_Value': results['p_value'],
    'Tau2_Between': results['tau2'],
    'Sigma2_Within': results['sigma2'],
    'I2_Total': i2_stats['i2_total'],
    'I2_Between': i2_stats['i2_between'],
    'I2_Within': i2_stats['i2_within']
}

summary_df = pd.DataFrame([summary_dict])

print("\n📊 RESULTS SUMMARY:")
print(summary_df.T)

# Save to CSV
# summary_df.to_csv('meta_analysis_results.csv', index=False)
# print("\n✅ Results saved to 'meta_analysis_results.csv'")

---

## 🎉 SUMMARY

This notebook demonstrated the complete `ecometa` workflow:

✅ **Data Loading** - Read and prepare meta-analysis data  
✅ **Three-Level Analysis** - `ThreeLevelMeta.fit()`  
✅ **Visualization** - Publication-ready plots (orchard, forest, funnel)  
✅ **Meta-Regression** - Explore moderators (linear and non-linear)  
✅ **Sensitivity Analysis** - Leave-one-out influence  
✅ **Publication Bias** - Funnel plots and diagnostics  
✅ **Model Comparison** - AIC/BIC for model selection  

**Key Advantages:**
- 🚀 **Fast**: No function definitions needed
- 📚 **Professional**: Matches R's metafor quality
- 🔬 **Rigorous**: REML, Sherman-Morrison, multi-start optimization
- 📊 **Beautiful**: Interactive and static publication-ready plots

---

### 📖 Next Steps

1. Replace the example data with your own dataset
2. Customize plots for publication
3. Explore additional moderators
4. Run subgroup analyses using `group_by` parameter

### 📚 Documentation

For detailed documentation, see:
- `help(ThreeLevelMeta)`
- `help(MetaRegression)`
- `help(forest_plot)`

---

**Happy Meta-Analyzing! 🌿**